## Checking Avilable GPU


In [ ]:
!nvidia-smi

## Importing requirements

In [ ]:
# pytorch libs
import torch
from torch import nn
import torchvision

# matplot libs
import matplotlib.pyplot as plt

# numpy
import numpy as np

# torch metrics
try:
  import torchmetrics
except:
  !pip3 -q install torchmetrics
try:
  import torchmetrics
except:
  !pip3 -q install torchmetrics
finally:
  from torchmetrics.classification import Accuracy

try:
  import torchinfo
except:
  !pip3 -q install torchinfo
finally:
  from torchinfo import summary

try:
  import torchattacks
except:
  !pip3 -q install torchattacks
finally:
  import torchattacks
 
import torch.nn.parallel
import torch.backends.cudnn as cudnn
# import torch.optim as optim
import torch.utils.data
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torchvision import datasets, transforms, models
from torchvision import utils as vutils

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from sklearn.model_selection import train_test_split

from timeit import default_timer as timer
from tqdm.auto import tqdm
import matplotlib.animation as animation
from IPython.display import HTML

## Device Agnostic code

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

## Getting Dataset

### Downloading

In [ ]:
!gdown 1uPnpDXpJLDY6JZSNpB9eYYIadmnS3upj

### Processing

In [ ]:
info = pd.read_csv("GAN_data.csv", sep = ',', )

In [ ]:
info.head()

In [ ]:
X, y = info.iloc[:,:1024].values, info.iloc[:, -1:].values

In [ ]:
print(f"X shape: {X.shape}")

In [ ]:
print(f"y shape: {y.shape}")

In [ ]:
img_data = []
for x in X:
  # np.expand_dims(x.reshape(32,32), axis=0)
  img_data.append(x.reshape(32,32))

img_data = np.array(img_data)

In [ ]:
# vis.
plt.imshow(img_data[torch.randint(len(img_data),(1,))].reshape(32,32), cmap = "gray")

plt.show()

In [ ]:
class_name = np.unique(y)

In [ ]:
class_name = list(class_name)
class_name

### label -> class idx

In [ ]:
for i in range(len(class_name)):
  y[y == class_name[i]] = i

In [ ]:
y

In [ ]:
# split data
x_train, x_test, y_train, y_test = train_test_split(img_data, y, test_size = 0.25, random_state = 64, shuffle = True)

# converting
y_train = y_train.astype(int)
y_test = y_test.astype(int)

up_x_train, up_x_test = [], []
for x in x_train:
  up_x_train.append(np.expand_dims(x, axis=0))

for x in x_test:
  up_x_test.append(np.expand_dims(x, axis=0))

up_x_train, up_x_test = np.array(up_x_train), np.array(up_x_test)

In [ ]:
up_x_train[0]

In [ ]:
train_data, test_data = [], []
for x,y in zip(up_x_train, y_train):
  x = torch.from_numpy(x)
  # print(y[0])
  y = int(y[0])
  x = x.type(torch.float)
  train_data.append((x,y))

for x,y in zip(up_x_test, y_test):
  x = torch.from_numpy(x)
  y = int(y[0])
  x = x.type(torch.float)
  test_data.append((x,y))

In [ ]:
class CustomTensorDataset(Dataset):
    """TensorDataset with support of transforms.
    """
    def __init__(self, tensors, transform=None):
        assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors)
        self.tensors = tensors
        self.transform = transform


    def __len__(self):
        return self.tensors[0].size(0)

    def __getitem__(self, index):
        x = self.tensors[0][index]

        if self.transform:
            x = self.transform(x)

        y = self.tensors[1][index]

        return x, y

In [ ]:
tensor_x_train = torch.Tensor(up_x_train) # transform to torch tensor
tensor_y_train = torch.Tensor(y_train)

tensor_x_test = torch.Tensor(up_x_test) # transform to torch tensor
tensor_y_test = torch.Tensor(y_test)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((64,64))
])

In [ ]:
train_dataset = CustomTensorDataset(tensors=(tensor_x_train, tensor_y_train), transform=transform)
test_dataset = CustomTensorDataset(tensors=(tensor_x_test, tensor_y_test), transform=transform)

### Converting dataset to dataloader

In [ ]:
BATCH_SIZE = 32
train_dataloader = DataLoader(dataset = train_dataset, batch_size = BATCH_SIZE, shuffle = True)
test_dataloader = DataLoader(dataset = test_dataset, batch_size = BATCH_SIZE, shuffle = False)

In [ ]:
print(f'Total {len(train_dataloader)} of train each of {BATCH_SIZE} batches.')
print(f'Total {len(test_dataloader)} of test each of {BATCH_SIZE} batches.')

## HyperParameters

In [ ]:
# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 32

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 1

# Size of z latent vector (i.e. size of generator input)
nz = 1

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 64

# Learning rate for optimizers
lr = 0.0003

# Beta1 hyperparameter for Adam optimizers
beta1 = 0.5

# number of gpu
ngpu = 1

In [ ]:
real_batch = next(iter(train_dataloader))
plt.figure(figsize=(13,13))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

## Weight INIT

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

## Generator

In [ ]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. ``(ngf*8) x 4 x 4``
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. ``(ngf*4) x 8 x 8``
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. ``(ngf*2) x 16 x 16``
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. ``(ngf) x 32 x 32``
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. ``(nc) x 64 x 64``
        )

    def forward(self, input):
        return self.main(input)

## Discriminator

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is ``(nc) x 64 x 64``
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf) x 32 x 32``
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf*2) x 16 x 16``
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf*4) x 8 x 8``
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf*8) x 4 x 4``
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

## Models

In [ ]:
# Create the generator
netG = Generator(ngpu).to(device)

# Handle multi-GPU if desired
if (device == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))

# Apply the ``weights_init`` function to randomly initialize all weights
#  to ``mean=0``, ``stdev=0.02``.
netG.apply(weights_init)

# Print the model
print(netG)

In [ ]:
# Create the Discriminator
netD = Discriminator(ngpu).to(device)

# Handle multi-GPU if desired
if (device == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))

# Apply the ``weights_init`` function to randomly initialize all weights
# like this: ``to mean=0, stdev=0.2``.
netD.apply(weights_init)

# Print the model
print(netD)

## Loss function and optimizer for GAN

In [ ]:
# Initialize the ``BCELoss`` function
criterion = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(32, nz, 1, 1, device=device)

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

# Setup Adam optimizers for both G and D
optimizerD = torch.optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = torch.optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
img_list = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")
# For each epoch
for epoch in tqdm(range(num_epochs)):
    # For each batch in the dataloader
    for i, data in enumerate(train_dataloader, 0):

        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        netD.zero_grad()
        # Format batch
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        # Forward pass real batch through D
        output = netD(real_cpu).view(-1)
        # Calculate loss on all-real batch
        errD_real = criterion(output, label)
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        # Generate fake image batch with G
        fake = netG(noise)
        label.fill_(fake_label)
        # Classify all fake batch with D
        output = netD(fake.detach()).view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, label)
        # Calculate the gradients for this batch, accumulated (summed) with previous gradients
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Compute error of D as sum over the fake and the real batches
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        # Calculate G's loss based on this output
        errG = criterion(output, label)
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()

        # Output training stats
        if i % 100 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch+1, num_epochs, i+1, len(train_dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(train_dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        iters += 1

## Save Model

In [ ]:
from pathlib import Path
def save_model(MODEL_NAME, model: torch.nn.Module):

  MODEL_PATH = Path("drive/MyDrive/Course/Sem2/DLOps/")
  MODEL_PATH.mkdir(parents = True, exist_ok = True)

  # MODEL_NAME = "ad_model.pth"
  MODEL_PATH_SAVE = MODEL_PATH / MODEL_NAME

  print(f"model saved at: {MODEL_PATH_SAVE}")
  torch.save(obj = model.state_dict(), f = MODEL_PATH_SAVE)

In [ ]:
save_model('netD_2.pth', netD)
save_model('netG_2.pth', netG)

In [ ]:
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())

In [ ]:
len(img_list)

In [ ]:
plt.imshow(np.transpose(img_list[-1],(1,2,0)))

In [ ]:
plt.imshow(train_dataset[1][0].permute(1,2,0), cmap = 'gray')

## Load model

In [ ]:
netG = Generator(ngpu).to(device)

# path
MODEL_PATH = Path("drive/MyDrive/Course/Sem2/DLOps/")

MODEL_NAME = "netG_2.pth"
MODEL_PATH_SAVE = MODEL_PATH / MODEL_NAME

if torch.cuda.is_available() == False:
  netG.load_state_dict(torch.load(f = MODEL_PATH_SAVE, map_location=torch.device('cpu')))
else:
  netG.load_state_dict(torch.load(f = MODEL_PATH_SAVE))

# model_18.state_dict()

netG = netG.to(device)

In [ ]:
netD = Discriminator(ngpu).to(device)

# path
MODEL_PATH = Path("drive/MyDrive/Course/Sem2/DLOps/")

MODEL_NAME = "netD_2.pth"
MODEL_PATH_SAVE = MODEL_PATH / MODEL_NAME

if torch.cuda.is_available() == False:
  netD.load_state_dict(torch.load(f = MODEL_PATH_SAVE, map_location=torch.device('cpu')))
else:
  netD.load_state_dict(torch.load(f = MODEL_PATH_SAVE))

# model_18.state_dict()

netD = netD.to(device)

In [ ]:
optimizerD = torch.optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = torch.optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
fixed_noise = torch.randn(1, nz, 1, 1, device=device)

In [ ]:
fake = netG(fixed_noise).detach().cpu()

In [ ]:
plt.imshow(fake[0].permute(1,2,0), cmap = 'gray')
print(fake)

In [ ]:
len(fake[0])

In [ ]:
train_dataset[1][0].unsqueeze(dim = 0).shape

In [ ]:
fake[0]

In [ ]:
# fixed_noise = torch.randn(1, nz, 1, 1, device=device)
fixed_noise = train_dataset[1][0].unsqueeze(dim = 0).to(device)
fake = netG(fixed_noise).detach().cpu()
plt.imshow(fake[0].permute(1,2,0), cmap = 'gray')

In [ ]:
import torchvision

In [ ]:
resize_fake = torchvision.transforms.functional.resize(fake[0], (64,64), interpolation=2)

In [ ]:
resize_fake.shape

In [ ]:
plt.imshow(resize_fake.permute(1,2,0), cmap = 'gray')

## Generative image datasets

In [ ]:
test_data = []
count = 0
count_list = [0 for i in range(len(class_name))]
# print(len(count_list))
i = 0
while count != 2300:
  fixed_noise = train_dataset[i][0].unsqueeze(dim = 0).to(device)
  label = train_dataset[i][1]
  # print(int(label.item()))
  fake = netG(fixed_noise).detach().cpu()
  resize_fake = torchvision.transforms.functional.resize(fake[0], (64,64), interpolation=2)
  # print(f"count: {count} and  label: {label} i: {i}")
  if count_list[int(label.item())] >= 50:
    i += 1
    continue
  else:
    test_data.append((resize_fake, label))
    count_list[int(label.item())] += 1
    count += 1
    i += 1

In [ ]:
len(test_data)

## make generated dataset

In [ ]:
new_test_data = torch.utils.data.ConcatDataset([test_data])

## dataloader

In [ ]:
BATCH_SIZE = 32

gen_test_dataloader = DataLoader(dataset = new_test_data, batch_size = BATCH_SIZE, shuffle = False)

## Train and Test Loop

In [ ]:
# Training Step
def training_step(
    model: nn.Module,
    dataloader: torch.utils.data.DataLoader,
    loss_fn: nn.Module,
    acc_fn: torchmetrics.classification.accuracy.MulticlassAccuracy,
    optimizer: torch.optim.Optimizer,
    device: str,
    profiler: torch.profiler.profile = None,
):
    """
    Desc:
      funtion to perform traning step for one EPOCH

    Args:
      model (nn.Module): Pytorch model class object
      dataloader (torch.utils.data.DataLoader): training dataloder from training dataset
      loss_fn (nn.Module): Loss Function (object) of your choice
      acc_fn (torchmetrics.classification.accuracy.MulticlassAccuracy): accuracy function from trochmetrics
      optimizer (torch.optim.Optimizer): Optimizer Function (object) of your choice
      device (str): Torch Device "CPU/GPU"
      profiler (torch.profiler.profile, optional): Pytorch Profiler. Defaults to None.
    Returns:
      train_loss (float), train_acc (float): training loss and training accuracy for one EPOCH
    """
    model.train()  # putting model in traing model

    train_loss, train_acc = 0, 0  # initlizing loss and acc. for the epoch

    if profiler != None:
      profiler.start()
      for step,(X, y) in enumerate(dataloader):  # loop in batches
        if step >= (1 + 1 + 3) * 2:
          break
        X, y = X.to(device), y.to(device)  # sending the data to target device
        # print(f"shape of X: {X.shape}, shape of y: {y.shape}")
        
        # 1. forward pass
        y_pred_logits = model(X)
        # y_pred = y_pred_logits.argmax(dim=1).type(torch.int)
        # print(y_pred)
        # 2. calculate the loss
        loss = loss_fn(y_pred_logits, y)
        train_loss += loss.item()

        # 3. optimizer zero grad
        optimizer.zero_grad()

        # 4. loss backword
        loss.backward()

        # 5. optimizer step
        optimizer.step()
        
        train_acc += acc_fn(y_pred_logits, y).item()
        
        profiler.step()
      profiler.stop()
    else:
      for step,(X, y) in enumerate(dataloader):  # loop in batches
        X, y = X.to(device), y.to(device)  # sending the data to target device
        # print(f"shape of X: {X.shape}, shape of y: {y.shape}")
        
        # 1. forward pass
        y_pred_logits = model(X)
        # y_pred = y_pred_logits.argmax(dim=1).type(torch.int)
        # print(y_pred)
        # 2. calculate the loss
        loss = loss_fn(y_pred_logits, y)
        train_loss += loss.item()

        # 3. optimizer zero grad
        optimizer.zero_grad()

        # 4. loss backword
        loss.backward()

        # 5. optimizer step
        optimizer.step()
        
        train_acc += acc_fn(y_pred_logits, y).item()
    # 6. returning actual loss and acc.x
    return train_loss / len(dataloader), train_acc / len(dataloader)

# Testing Step
def testing_step(
    model: nn.Module,
    dataloader: torch.utils.data.DataLoader,
    loss_fn: nn.Module,
    acc_fn: torchmetrics.classification.accuracy.MulticlassAccuracy,
    device: str,
):
    """
    Desc:
      funtion to perform testing step for one EPOCH

    Args:
      model (nn.Module): Pytorch model class object
      dataloader (torch.utils.data.DataLoader): testing dataloder from training dataset
      loss_fn (nn.Module): Loss Function (object) of your choice
      acc_fn (torchmetrics.classification.accuracy.MulticlassAccuracy): accuracy function from trochmetrics
      device (str): Torch Device "CPU/GPU"
    Returns:
      test_loss (float), test_acc (float): testing loss and testing accuracy for one EPOCH
    """
    model.eval()  # putting model in eval model

    test_loss, test_acc = 0, 0  # initlizing loss and acc. for the epoch

    # with torch.inference_mode(): # disabling inference mode for aqcuiring gradients of perturbed data
    for (X, y) in dataloader:  # loop in batches
        X, y = X.to(device), y.to(device)  # sending the data to target device
        # print(f"shape of X: {X.shape}, shape of y: {y.shape}")

        # 1. forward pass
        y_pred_logits = model(X)

        # 2. calculate the loss
        loss = loss_fn(y_pred_logits, y)
        test_loss += loss.item()

        # printing the prediction and actual label
        # print(y_pred_logits.argmax(dim=1), y,sep='\n')
        
        # 3. calculating accuracy
        test_acc += acc_fn(y_pred_logits, y).item()

    # 6. returning actual loss and acc.
    return test_loss / len(dataloader), test_acc / len(dataloader)


# Training Function
from tqdm.auto import tqdm


def train(
    model: nn.Module,
    train_dataloader: torch.utils.data.DataLoader,
    test_dataloader: torch.utils.data.DataLoader,
    loss_fn: nn.Module,
    acc_fn: torchmetrics.classification.accuracy.MulticlassAccuracy,
    optimizer: torch.optim.Optimizer,
    epochs: int,
    save_info: str,
    profiler:torch.profiler.profile = None,
):
    """
    Desc:
      funtion to perform training for one {epochs} param

    Args:
      model (nn.Module): Pytorch model class object
      train_dataloader (torch.utils.data.DataLoader): training dataloder from training dataset
      test_dataloader (torch.utils.data.DataLoader): testing dataloder from testing dataset
      loss_fn (nn.Module): Loss Function (object) of your choice
      acc_fn (torchmetrics.classification.accuracy.MulticlassAccuracy): accuracy function from trochmetrics
      device (str): Torch Device "CPU/GPU"
      epochs (int): number of epochs to train the model
      save_info (str): info for saving model
      profiler (torch.profiler.profile, optional): Pytorch Profiler. Defaults to None.
    Returns:
      result_dict ({str: List[float]}): testing loss and testing accuracy for one EPOCH
    """
    # 1. init. result dict
    result_dict = {"train_loss": [], "train_acc": [], "test_loss": [], "test_acc": []}
    best_test_acc = 0
    best_model_params = None

    # 2. loop in range epochs performs train and test steps on the model
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = training_step(
            model=model,
            dataloader=train_dataloader,
            loss_fn=loss_fn,
            acc_fn=acc_fn,
            optimizer=optimizer,
            device=device,
            profiler=profiler,
        )

        test_loss, test_acc = testing_step(
            model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn,
            acc_fn=acc_fn,
            device=device,
        )

        # 3. print out results
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        # 4. adding results to result_dict
        result_dict["train_loss"].append(train_loss)
        result_dict["train_acc"].append(train_acc)
        result_dict["test_loss"].append(test_loss)
        result_dict["test_acc"].append(test_acc)
        
        if test_acc > best_test_acc:
            best_test_acc = test_acc
            best_model_params = model.state_dict()

    # 5. saving the best model
    torch.save(best_model_params, f"{model.__class__.__name__}_epoch_{epochs}_optim_adam_"+save_info+".pth")
    # 6. return result dict
    return result_dict

## Plot graph

In [ ]:
def plot_graph(train_losses, test_losses, train_accs, test_accs):
  plt.figure(figsize = (20, 8))
  plt.subplot(1, 2, 1)
  plt.plot(range(len(train_losses)), train_losses, label = "Train Loss")
  plt.plot(range(len(test_losses)), test_losses, label = "Test Loss")
  plt.legend()
  plt.xlabel("Epoches")
  plt.ylabel("Loss")
  # plt.show()

  plt.subplot(1, 2, 2)
  plt.plot(range(len(train_accs)), train_accs, label = "Train Accuracy")
  plt.plot(range(len(test_accs)), test_accs, label = "Test Accuracy")
  plt.legend()
  plt.xlabel("Epoches")
  plt.ylabel("Accuracy")
  plt.show()

## Load model

In [ ]:
model_18 = models.resnet18(weights = models.ResNet18_Weights.IMAGENET1K_V1, progress = False).to(device)
model_18.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False).to(device)
feature_number = model_18.fc.in_features
model_18.fc = nn.Linear(feature_number, len(class_name)).to(device)

model_18

## Loss and Accuracy Function

In [ ]:
# loss and accuracy function
loss_fn = nn.CrossEntropyLoss()

accuracy_fn = Accuracy("multiclass",num_classes = len(class_name)).to(device)

## Train resnet18

In [ ]:
# init. epochs
epoches = 15


model_18 = models.resnet18(weights = 'DEFAULT', progress = False)
model_18.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
feature_number = model_18.fc.in_features
model_18.fc = nn.Linear(feature_number, len(class_name))
model_18 = model_18.to(device)


optimizer = torch.optim.Adam(params = model_18.parameters(), lr = 1e-3)

model_18_train_loss, model_18_test_loss = [], []
model_18_train_accs, model_18_test_accs = [], []
print()

start_time = timer()
torch.manual_seed(64)
torch.cuda.manual_seed(64)
for epoch in tqdm(range(epoches)):
  print(f"Epoch: {epoch+1}")
  train_loss, train_acc = training_step(
      model = model_18,
      dataloader = train_dataloader,
      loss_fn = loss_fn,
      optimizer = optimizer,
      accuracy_fn = accuracy_fn,
      device = device
      )
  
  test_loss, test_acc = testing_step(
      model = model_18, 
      dataloader = gen_test_dataloader,
      loss_fn = loss_fn,
      accuracy_fn = accuracy_fn,
      device = device
      )
  
  model_18_train_loss.append(train_loss.item())
  model_18_test_loss.append(test_loss.item())
  model_18_train_accs.append(train_acc.item())
  model_18_test_accs.append(test_acc.item())


  print(f"Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f} | Train Accuray: {train_acc:.4f} | Test Accuracy: {test_acc:.4f}")
  print()

plot_graph(model_18_train_loss, model_18_test_loss, model_18_train_accs, model_18_test_accs)

end_time = timer()

print(f"Execution time: {end_time - start_time} Seconds.")

## F1 Score

In [ ]:
from torchmetrics.classification import MulticlassF1Score


y_preds = []
target = []
with torch.inference_mode():
  for x, y in tqdm(gen_test_dataloader, desc = "Making prediction..."):
    y = y.squeeze()
    y = y.type(torch.LongTensor)
    x, y = x.to(device), y.to(device)

    logit = model_18(x)
    pred = torch.softmax(logit.squeeze(), dim = 1).argmax(dim = 1)

    y_preds.append(pred)
    target.append(y)

y_tensor_preds = torch.cat(y_preds).to(device)
target_val = torch.cat(target).to(device)


metric = MulticlassF1Score(len(class_name)).to(device)
metric(target_val, y_tensor_preds)


## Confusion Matrix

In [ ]:
from torchmetrics import ConfusionMatrix
from mlxtend.plotting import plot_confusion_matrix

confmat = ConfusionMatrix(num_classes = len(class_name), task = 'multiclass')
confmat_tensor = confmat(preds = y_tensor_preds.cpu(), target = target_val.cpu())

fix, ax = plot_confusion_matrix(conf_mat = confmat_tensor.numpy(), figsize = (13,13))